In [2]:
# imodels imports
from imodels import get_clean_dataset
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusClassifier, RandomForestPlusRegressor
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import  AloRFPlusMDI, RFPlusMDI

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, root_mean_squared_error

# other important libraries
import numpy as np
import pandas as pd
import xgboost as xgb

In [8]:
genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_esm_prod_pppl_full_ptv.csv")
genotype

,ENSG00000155657_esm_prod_0,ENSG00000155657_esm_prod_1,ENSG00000155657_esm_prod_2,ENSG00000155657_esm_prod_3,ENSG00000155657_esm_prod_4,ENSG00000155657_esm_prod_5,ENSG00000155657_esm_prod_6,ENSG00000155657_esm_prod_7,ENSG00000155657_esm_prod_8,ENSG00000155657_esm_prod_9,...,2,3,4,5,6,7,8,9,10,11
0,1.159392e+06,53.150624,-6.947658,14.544866,11.080802,-15.187052,-3.704933,-2.172298,4.941594,-3.851102,...,-14.8232,6.34158,-0.149284,3.079470,-2.65142,-1.422720,2.519350,-1.354790,-0.105849,5.789740
1,-8.322584e+05,3.609315,6.410834,3.324862,1.820494,3.346366,6.417796,-0.604631,4.626436,-0.680389,...,-12.5360,6.91117,-1.549940,-1.794670,-4.96364,-3.814100,5.289460,-3.050990,-4.269630,-0.530574
2,1.159389e+06,52.004866,-8.190168,9.668794,18.887932,-14.371820,-2.844055,1.600584,4.930808,7.875503,...,-15.6902,5.19352,-2.030970,1.857810,-1.04143,-2.310750,-1.988220,-1.379210,5.147300,-2.660630
3,5.618885e+05,37.707810,-3.088684,8.799338,9.729975,13.299167,-67.939636,8.422758,0.444249,4.591365,...,-14.5245,4.66729,-1.694440,6.084100,1.51288,-2.057870,0.984135,-1.844740,-2.955540,2.409810
4,2.354371e+06,-151.839066,-67.670308,-51.438584,-35.715556,55.223306,21.907905,28.701004,0.724605,-35.293530,...,-11.8629,1.19131,-0.691440,2.401740,-5.41882,0.195783,0.113224,-2.276820,7.193060,6.534780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,1.635653e+05,32.338714,-29.068675,-86.761271,1.270804,32.215282,15.112363,32.807135,2.062718,-5.997158,...,-11.4531,3.87597,-1.390360,4.543550,-2.97738,-0.566625,-0.319262,0.345175,-5.943890,1.634150
22538,-4.339222e+05,13.135232,7.188963,13.751606,-2.757324,-8.670328,-0.275546,-15.238093,3.813261,-3.097620,...,4.4830,3.87090,5.963360,-20.854400,2.56826,2.250370,12.044400,1.293590,-3.276760,-12.048100
22539,2.155207e+06,70.835393,20.425844,37.773548,-39.207414,27.791084,13.316613,-2.625096,4.529860,-29.523612,...,-13.1914,3.42565,-1.183190,3.156370,-1.35358,-0.036006,-1.449080,-1.900950,3.274970,-0.588565
22540,-3.559524e+04,26.368837,-14.866167,-53.600477,-29.908735,-10.010740,-7.646984,-71.552435,-0.651454,1.425238,...,-9.1427,3.15598,-2.184210,-3.060840,-7.26838,-3.319090,-2.294830,-4.789090,3.248110,2.456600


In [ ]:
genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_oh.csv")
# genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_esm_prod_pppl_full_ptv.csv")
phenotype = pd.read_csv('/scratch/users/omer_ronen/mutemb_esm/y_ilvm_oh.csv')
# make genotype a numpy array
genotype = genotype.to_numpy()
# get last twelve features in genotype
limited_genotype = genotype[:,-12:]
# make phenotype a 1D numpy array
phenotype = phenotype.to_numpy().reshape(-1)

# train-test split
genotype_train, genotype_test, limited_genotype_train, \
limited_genotype_test, phenotype_train, phenotype_test = \
    train_test_split(genotype, limited_genotype, phenotype, test_size = 0.3,
                     random_state = 1)
        
# fit random forest model
rf = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state = 42)
rf.fit(genotype_train, phenotype_train)
rf_limited = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state = 42)
rf_limited.fit(limited_genotype_train, phenotype_train)

# predict on test set
phenotype_pred = rf.predict(genotype_test)
phenotype_pred_limited = rf_limited.predict(limited_genotype_test)

# calculate correlation for pred-check
cor = np.corrcoef(phenotype_test, phenotype_pred)[0,1]
cor_limited = np.corrcoef(phenotype_test, phenotype_pred_limited)[0,1]

print("Full Model Correlation: ", cor)
print("Limited Model Correlation: ", cor_limited)

Full Model Correlation:  0.6307915897423984
Limited Model Correlation:  0.6302596772658053


In [6]:
genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_oh.csv")
# genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_esm_prod_pppl_full_ptv.csv")
phenotype = pd.read_csv('/scratch/users/omer_ronen/mutemb_esm/y_ilvm_oh.csv')
# make genotype a numpy array
genotype = genotype.to_numpy()
# get last twelve features in genotype
limited_genotype = genotype[:,-12:]
# make phenotype a 1D numpy array
phenotype = phenotype.to_numpy().reshape(-1)

cor_limited = []
cor_full = []
r2_limited = []
r2_full = []
rmse_limited = []
rmse_full = []

for i in range(20):
    # train-test split
    genotype_train, genotype_test, limited_genotype_train, \
    limited_genotype_test, phenotype_train, phenotype_test = \
        train_test_split(genotype, limited_genotype, phenotype, test_size = 0.3,
                        random_state = i)
    # fit random forest model
    rf = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state = 42)
    rf.fit(genotype_train, phenotype_train)
    rf_limited = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state = 42)
    rf_limited.fit(limited_genotype_train, phenotype_train)

    # predict on test set
    phenotype_pred = rf.predict(genotype_test)
    phenotype_pred_limited = rf_limited.predict(limited_genotype_test)

    # calculate correlation for pred-check
    cor_full.append(np.corrcoef(phenotype_test, phenotype_pred)[0,1])
    cor_limited.append(np.corrcoef(phenotype_test, phenotype_pred_limited)[0,1])
    r2_full.append(r2_score(phenotype_test, phenotype_pred))
    r2_limited.append(r2_score(phenotype_test, phenotype_pred_limited))
    rmse_full.append(root_mean_squared_error(phenotype_test, phenotype_pred))
    rmse_limited.append(root_mean_squared_error(phenotype_test, phenotype_pred_limited))
    
cor_full = np.array(cor_full)
cor_limited = np.array(cor_limited)
r2_full = np.array(r2_full)
r2_limited = np.array(r2_limited)
rmse_full = np.array(rmse_full)
rmse_limited = np.array(rmse_limited)
# get means and sds of each
mean_cor_full = np.mean(cor_full)
mean_cor_limited = np.mean(cor_limited)
sd_cor_full = np.std(cor_full)
sd_cor_limited = np.std(cor_limited)
mean_r2_full = np.mean(r2_full)
mean_r2_limited = np.mean(r2_limited)
sd_r2_full = np.std(r2_full)
sd_r2_limited = np.std(r2_limited)
mean_rmse_full = np.mean(rmse_full)
mean_rmse_limited = np.mean(rmse_limited)
sd_rmse_full = np.std(rmse_full)
sd_rmse_limited = np.std(rmse_limited)

print("Full Model Correlation: ", mean_cor_full, " +/- ", sd_cor_full)
print("Limited Model Correlation: ", mean_cor_limited, " +/- ", sd_cor_limited)
print("Minimum Difference in Correlation: ", np.min(cor_full - cor_limited))
print("Min of Full Correlation - Max of Limited Correlation: ", np.min(cor_full) - np.max(cor_limited))

print("Full Model R2: ", mean_r2_full, " +/- ", sd_r2_full)
print("Limited Model R2: ", mean_r2_limited, " +/- ", sd_r2_limited)
print("Minimum Difference in R2: ", np.min(r2_full - r2_limited))
print("Min of Full R2 - Max of Limited R2: ", np.min(r2_full) - np.max(r2_limited))

print("Full Model RMSE: ", mean_rmse_full, " +/- ", sd_rmse_full)
print("Limited Model RMSE: ", mean_rmse_limited, " +/- ", sd_rmse_limited)
print("Minimum Difference in RMSE: ", np.min(rmse_full - rmse_limited))
print("Min of Full RMSE - Max of Limited RMSE: ", np.min(rmse_full) - np.max(rmse_limited))

Full Model Correlation:  0.6283474015852735  +/-  0.004913913615241869
Limited Model Correlation:  0.6282689917860866  +/-  0.004970445344235337
Minimum Difference in Correlation:  -0.0005021615925988598
Min of Full Correlation - Max of Limited Correlation:  -0.01750065169358539
Full Model R2:  0.39437944691740767  +/-  0.006239891379850461
Limited Model R2:  0.3942764052299147  +/-  0.006313129672735268
Minimum Difference in R2:  -0.00068856590086952
Min of Full R2 - Max of Limited R2:  -0.022201831451174714
Full Model RMSE:  6.643785953975964  +/-  0.0812864149692228
Limited Model RMSE:  6.644335398139373  +/-  0.08032920158349648
Minimum Difference in RMSE:  -0.0043769474032338
Min of Full RMSE - Max of Limited RMSE:  -0.2641286077247198


In [9]:
# genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_oh.csv")
# genotype = pd.read_csv("/scratch/users/omer_ronen/mutemb_esm/X_k_5_ilvm_esm_prod_pppl_full_ptv.csv")
phenotype = pd.read_csv('/scratch/users/omer_ronen/mutemb_esm/y_ilvm_oh.csv')
# make genotype a numpy array
genotype = genotype.to_numpy()
# get last twelve features in genotype
limited_genotype = genotype[:,-12:]
# make phenotype a 1D numpy array
phenotype = phenotype.to_numpy().reshape(-1)

cor_limited = []
cor_full = []
r2_limited = []
r2_full = []
rmse_limited = []
rmse_full = []

for i in range(1):
    # train-test split
    genotype_train, genotype_test, limited_genotype_train, \
    limited_genotype_test, phenotype_train, phenotype_test = \
        train_test_split(genotype, limited_genotype, phenotype, test_size = 0.3,
                        random_state = i)
    # fit xgboost on full and limited models
    params = {
        'objective': 'reg:squarederror',  # Use 'reg:squarederror' for regression
        'max_depth': 5,
        'eta': 0.1,
        'eval_metric': 'logloss'
    }
    limited_dtrain = xgb.DMatrix(limited_genotype_train, label=phenotype_train)
    full_dtrain = xgb.DMatrix(genotype_train, label=phenotype_train)
    # train the model
    limited_model = xgb.train(params, limited_dtrain, num_boost_round=100)
    full_model = xgb.train(params, full_dtrain, num_boost_round=100)
    # predict on test set
    limited_pred = limited_model.predict(xgb.DMatrix(limited_genotype_test))
    full_pred = full_model.predict(xgb.DMatrix(genotype_test))
    # calculate correlation for pred-check
    cor_full.append(np.corrcoef(phenotype_test, full_pred)[0,1])
    cor_limited.append(np.corrcoef(phenotype_test, limited_pred)[0,1])
    r2_full.append(r2_score(phenotype_test, full_pred))
    r2_limited.append(r2_score(phenotype_test, limited_pred))
    rmse_full.append(root_mean_squared_error(phenotype_test, full_pred))
    rmse_limited.append(root_mean_squared_error(phenotype_test, limited_pred))

In [10]:
cor_full = np.array(cor_full)
cor_limited = np.array(cor_limited)
r2_full = np.array(r2_full)
r2_limited = np.array(r2_limited)
rmse_full = np.array(rmse_full)
rmse_limited = np.array(rmse_limited)
# get means and sds of each
mean_cor_full = np.mean(cor_full)
mean_cor_limited = np.mean(cor_limited)
sd_cor_full = np.std(cor_full)
sd_cor_limited = np.std(cor_limited)
mean_r2_full = np.mean(r2_full)
mean_r2_limited = np.mean(r2_limited)
sd_r2_full = np.std(r2_full)
sd_r2_limited = np.std(r2_limited)
mean_rmse_full = np.mean(rmse_full)
mean_rmse_limited = np.mean(rmse_limited)
sd_rmse_full = np.std(rmse_full)
sd_rmse_limited = np.std(rmse_limited)

print("Full Model Correlation: ", mean_cor_full, " +/- ", sd_cor_full)
print("Limited Model Correlation: ", mean_cor_limited, " +/- ", sd_cor_limited)
print("Minimum Difference in Correlation: ", np.min(cor_full - cor_limited))
print("Min of Full Correlation - Max of Limited Correlation: ", np.min(cor_full) - np.max(cor_limited))

print("Full Model R2: ", mean_r2_full, " +/- ", sd_r2_full)
print("Limited Model R2: ", mean_r2_limited, " +/- ", sd_r2_limited)
print("Minimum Difference in R2: ", np.min(r2_full - r2_limited))
print("Min of Full R2 - Max of Limited R2: ", np.min(r2_full) - np.max(r2_limited))

print("Full Model RMSE: ", mean_rmse_full, " +/- ", sd_rmse_full)
print("Limited Model RMSE: ", mean_rmse_limited, " +/- ", sd_rmse_limited)
print("Minimum Difference in RMSE: ", np.min(rmse_full - rmse_limited))
print("Min of Full RMSE - Max of Limited RMSE: ", np.min(rmse_full) - np.max(rmse_limited))

Full Model Correlation:  0.6169481551089033  +/-  0.0
Limited Model Correlation:  0.6205696145882396  +/-  0.0
Minimum Difference in Correlation:  -0.003621459479336231
Min of Full Correlation - Max of Limited Correlation:  -0.003621459479336231
Full Model R2:  0.37968110494361884  +/-  0.0
Limited Model R2:  0.3839563913017795  +/-  0.0
Minimum Difference in R2:  -0.004275286358160657
Min of Full R2 - Max of Limited R2:  -0.004275286358160657
Full Model RMSE:  6.708001326981556  +/-  0.0
Limited Model RMSE:  6.684845324765358  +/-  0.0
Minimum Difference in RMSE:  0.02315600221619807
Min of Full RMSE - Max of Limited RMSE:  0.02315600221619807


In [ ]:
# train a rf regressor
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print(f'R^2 on Test Data: {r2_score(y_test, rf_pred)}')

R^2 on Test Data: 0.39809163318237584


In [4]:
# fit rf+
rf_plus = RandomForestPlusRegressor(rf_model = rf, prediction_model = Ridge())
rf_plus.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  5.6min finished


In [5]:
# create an explainer
lmdi_explainer = RFPlusMDI(rf_plus)
# calculate feature importance
lmdi_values = lmdi_explainer.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True, normalize=True)

KeyboardInterrupt: 